In [ ]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 KB 6.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.5/534.5 KB 11.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 8.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 KB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 KB 4.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 26.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47

In [10]:
import os

os.environ['HF_HOME'] = '/local/scratch/tkim462/.cache/huggingface'
os.environ['XDG_CACHE_HOME'] = '/local/scratch/tkim462/.cache'

In [11]:
NUM_QUERIES = 100

In [16]:
from datasets import load_dataset

dataset = load_dataset("microsoft/ms_marco", 'v1.1')
dataset['validation'][0]

{'answers': ['Approximately $15,000 per year.'],
 'passages': {'is_selected': [1, 0, 0, 0, 0, 0],
  'passage_text': ['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.',
   'The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).',
   'In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgr

In [14]:
print(dataset['validation'][1])

if NUM_QUERIES is not None:
    dataset['validation'] = dataset['validation'].select(range(min(NUM_QUERIES, len(dataset['validation']))))

{'answers': ['$21,550 per year', 'The average hourly wage for a bartender is $10.36 and the average yearly take-home is $21,550.'], 'passages': {'is_selected': [0, 1, 0, 0, 0, 0, 0, 0], 'passage_text': ['A bartender’s income is comprised mostly of tips– 55% to be exact. In some states, employers aren’t even required to pay their bartenders the minimum wage and can pay as low as $2.13 per hour, and they depend on their tips almost entirely. Bartending can be a lot of things. For some it is exciting, for others exhausting. At times there is a lot of fun to be had, at others it is rather dull. But for the most part, bartending is almost always rewarding in the financial sense, as long as you stick with it.', 'According to the Bureau of Labor Statistics, the average hourly wage for a bartender is $10.36, and the average yearly take-home is $21,550. Bartending can be a lot of things. For some it is exciting, for others exhausting. At times there is a lot of fun to be had, at others it is ra

In [ ]:
def get_prefix_prompt(query, num):
    return [
        {'role': 'system', 'content': "You are RankGPT, an intelligent assistant that can rank passages based on their relevancy to the query."},
        {'role': 'user', 'content': f"I will provide you with {num} passages, each indicated by number identifier []. Rank the passages based on their relevance to query: {query}."},
        {'role': 'assistant', 'content': 'Okay, please provide the passages.'}
    ]

def get_post_prompt(query, num):
    # Simple list output rather than the > format, easier to parse 
    return (f"Search Query: {query}. \n"
            f"Rank the {num} passages above based on their relevance to the search query. "
            f"The most relevant passages should be listed first. \n"
            f"Output the ranked identifiers as a comma-separated list inside brackets, "
            f"for example: [3, 0, 5, ...]. \n"
            f"Only response the ranking results, do not say any word or explain.")
    
def remove_duplicate(response):
    new_response = []
    for c in response:
        if c not in new_response:
            new_response.append(c)
    return new_response

def create_ranking_prompt(item):
    num = len(item['passages']['is_selected'])
    query = item['query']
    post_prompt = get_post_prompt(query, num)
    
    # Combine prefix, passages, and post prompt
    messages = get_prefix_prompt(query, num)
    for i, passage in enumerate(item['passages']['passage_text'], start=1):
        messages.append({'role': 'user', 'content': f'[{i}] {passage}'})
        messages.append({'role': 'assistant', 'content': f'Received passage [{i}].'})
    messages.append({'role': 'user', 'content': post_prompt})
    
    return messages

In [17]:
from vllm import LLM, SamplingParams

# 1. Initialize the model on your H200
# For Llama-3-8B or 70B
model_name = "meta-llama/Meta-Llama-3-8B-Instruct" 
llm = LLM(model=model_name, tensor_parallel_size=1) # Use tensor_parallel_size=4 or 8 for 70B if needed

# 2. Define Sampling Parameters (Greedy decoding for ranking)
sampling_params = SamplingParams(temperature=0, max_tokens=100)

# 3. Generate prompts for a batch of MS MARCO items
# Let's say 'dataset' is your loaded MS MARCO split
conversations = [create_ranking_prompt(item) for item in dataset['validation']]

# 4. Run Batch Inference
outputs = llm.chat(messages=conversations, sampling_params=sampling_params, use_tqdm=True)

# Extract raw text results
raw_results = [output.outputs[0].text for output in outputs]

INFO 01-15 05:49:45 [importing.py:44] Triton is installed but 0 active driver(s) found (expected 1). Disabling Triton to prevent runtime errors.
INFO 01-15 05:49:45 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.
INFO 01-15 05:49:46 [utils.py:253] non-default args: {'disable_log_stats': True, 'model': 'meta-llama/Meta-Llama-3-8B-Instruct'}


RuntimeError: Device string must not be empty